In [1]:
%load_ext autoreload
%autoreload 2

import asyncio
import pandas as pd
from meeting_search import VectorSearch, build_context_string
from vexa import VexaAPI
import pandas as pd

# Initialize the VexaAPI
vexa = VexaAPI()
from core import generic_call, system_msg, user_msg
from pydantic_models import Summary

User information retrieved successfully.


In [2]:


def create_exploded_points_df(output):
    points_data = output[0].model_dump()['points']
    df_points = pd.DataFrame(points_data)
    df_points = df_points.rename(columns={'c': 'point'})
    def create_range(start, end):
        return list(range(int(start), int(end) + 1))
    df_points['range'] = df_points.apply(lambda row: create_range(row['s'], row['e']), axis=1)
    df_exploded = df_points.explode('range')
    df_exploded = df_exploded.drop(columns=['s', 'e'])
    df_exploded = df_exploded.reset_index(drop=True)
    return df_exploded

In [3]:
def combine_initials_and_content(group):
    combined = group['speaker'].fillna('').astype(str) + ': ' + group['content'].fillna('')


    return ' '.join(combined)

In [4]:
from prompts import Prompts
prompts = Prompts()

In [10]:
analyzer = VectorSearch(gpu_device=3)
meetings = vexa.get_meetings()
meetings = meetings[-200:]


In [11]:
meetings

[{'id': '9845b8ca-b2a0-4371-8a9a-daa270ec7ab7',
  'title': 'call_2024-08-29T09:17:13+00:00',
  'start_timestamp': '2024-08-29T09:17:13Z',
  'finish_timestamp': '2024-08-29T09:41:24.875999Z',
  'meeting': {'id': 'cd81c41a-0eba-4021-93e2-f740d95df46e',
   'external_id': 'qyo-vkuc-ubb',
   'title': None}},
 {'id': '2ef8bcc3-d3fd-473a-a7c8-865873cb038c',
  'title': 'call_2024-08-29T10:11:21+00:00',
  'start_timestamp': '2024-08-29T10:11:21Z',
  'finish_timestamp': '2024-08-29T10:41:49.960000Z',
  'meeting': {'id': '85f9c1e3-c299-48a2-a660-2a77b8af8991',
   'external_id': 'csx-mskk-qyk',
   'title': None}},
 {'id': '5dfa0c1d-6dfc-49db-9d56-5914ad21ef3d',
  'title': 'call_2024-08-29T13:13:16+00:00',
  'start_timestamp': '2024-08-29T13:13:16Z',
  'finish_timestamp': '2024-08-29T13:30:06.056000Z',
  'meeting': {'id': 'a023f3d6-5966-477a-b314-66c6835a15af',
   'external_id': 'zkr-qgkq-sio',
   'title': None}},
 {'id': 'df0a9f4c-3b7b-436b-a329-504b99cfe37d',
  'title': 'call_2024-08-29T18:25:59+

In [12]:
vexa.user_id

'ef7c085b-fdb5-4c94-b7b6-a61a3d04c210'

In [13]:
vexa.user_name

'Dmitriy Grankin'

In [14]:
#analyzer.delete_collection()

In [15]:
for meeting in meetings:
    if meeting['finish_timestamp']:
        meeting_id = meeting['id']
        if not analyzer.check_meeting_session_id_exists(meeting_id):
            result = vexa.get_transcription(meeting_session_id=meeting_id,use_index=True)
            if result:
                df, formatted_input, start_datetime, speakers = result
                output = await Summary.call([system_msg(prompts.think+ prompts.summarize_meeting+ '.The User: '+ vexa.user_name),user_msg(formatted_input)],model='gpt-4o-mini')
                df_exploded = create_exploded_points_df(output)
                joined_df = df_exploded.join(df, on='range', rsuffix='_transcript')

                points_with_qoutes = joined_df.groupby('point').apply(combine_initials_and_content)
                points_with_qoutes.name = 'qoutes'
                points_with_qoutes = points_with_qoutes.reset_index().to_dict(orient='records')
                summary = output[0].summary
                meeting_name = output[0].meeting_name
                
                if points_with_qoutes:
                    chunks = [f"{summary}\n\n{p['point']}\n\n{p['qoutes']}" for p in points_with_qoutes]
                    points = [p['point'] for p in points_with_qoutes]
                    qoutes = [p['qoutes'] for p in points_with_qoutes]
                    await analyzer.add_summary(meeting_name, summary, start_datetime, speakers, meeting_id,vexa.user_id,vexa.user_name)
                    await analyzer.update_vectorstore_with_qoutes(chunks, points, qoutes, start_datetime, speakers, meeting_id,vexa.user_id,vexa.user_name)
